In [ ]:
import pandas as pd
import numpy as np
from math import log
from numpy import dot
from numpy.linalg import norm

In [ ]:
detail_data = pd.read_csv("webtoon_detail.csv",names =['웹툰제목','점수','작가','장르','추가정보','링크'])
add_num = list(range(344))
detail_data["num"] = add_num

In [ ]:
vec_data = pd.read_csv("vec_data.csv",names =['댓글'])#csv파일을 불러와 사용가능하도록 처리하는 과정입니다.
vec_data= vec_data.fillna('')
vec_array_data = np.array(vec_data['댓글'])
vec_array_list = []
for i in range(len(vec_array_data)): # 선택한 댓글을 하나의 리스트 변경
        vec_array_list.append(vec_array_data[i])

In [ ]:
def title_num(code): #원하는 웹툰의 순서를 알아내는 함수
    detail_data = pd.read_csv("webtoon_detail.csv",names =['웹툰제목','점수','작가','장르','추가정보','링크'])
    add_num = list(range(344))
    detail_data["num"] = add_num
    select_data = detail_data[detail_data.index == code]
    num = int(select_data['num'])
    return num

In [ ]:
def tf(term, document): # tf-idf vectorizer 함수
    return document.count(term)

def idf(term):
    df = 0
    for doc in vec_array_list:
        if term in doc:
            df = df + 1
    return log(len(vec_array_list)/(df+1))+1

def tfidf(term, document):
    return tf(term, document)*idf(term)

def tfidf_list(text, keywords_list):
    tfidf_list = []
    for doc in text:
        tfidf_list.append([])
        for token in keywords_list:
            tfidf_list[-1].append(tfidf(token, doc))

    tfidf_pd = pd.DataFrame(tfidf_list, columns=keywords_list)# 각 키워드가 문장에 들어있는지 데이터 베이스화
    return tfidf_list, tfidf_pd

In [ ]:
def cos_sim(A, B): #코사인 유사도를 구하는 함수
       return dot(A, B)/(norm(A)*norm(B))

def cos_sim_index(tfidf_list, tfidf_pd, num): #요청한 웹툰과 코사인 유사도가 가장 높은 순서를 찾아내는 함수입니다.
    cos_sim_list = []
    for i in range(len(tfidf_list)):
        if i == num:
            cos_sim_list.append(0)
        else:
            cos_sim_list.append(cos_sim(tfidf_pd.values[num], tfidf_pd.values[i]))
        
    max_num = cos_sim_list.index(max(cos_sim_list))
    i_index = max_num
    if i_index == 0:
        print('유사 모델이 없습니다.')
        return 'none'
    else:
        print(num,'번과 ',i_index,'번이 가장 유사 모델입니다.')
        print('코사인 유사도:', cos_sim(tfidf_pd.values[num], tfidf_pd.values[i_index]))
        return i_index

In [33]:
input_code = input()
num = (title_num(input_code) - 1)
from nltk.tokenize import WordPunctTokenizer

token_list = WordPunctTokenizer().tokenize(vec_array_list[num])
sw = ['.', ',', "'"]

sw_removed = []
for i in token_list:
    if i.lower() not in sw:
        sw_removed.append(i)

vector_list, vector_pd = tfidf_list(vec_array_list,sw_removed)
find_num = cos_sim_index(vector_list, vector_pd, num)
find_web_code = detail_data[detail_data['num'] == find_num].index.tolist()
print(find_web_code)

733766
0 번과  136 번이 가장 유사 모델입니다.
코사인 유사도: 0.5168426649495261
['736281']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
